# Memory Analysis Deeper Dive

For this lab, I will do the following

* Identify suspicious processes
* Investigate their handles
* Investigate their privileges
* Investigate their PE version
* Investigate any additional IOC

# Suspicious process identification

I will start by running volatility3 and getting system information, followed by listing the processes that were running on the machine

In [2]:
cd ~\Desktop

In [3]:
cd volatility3

In [4]:
pwd


Path
----
C:\Users\anon\Desktop\volatility3



In [5]:
python .\vol.py -q -f ..\APT\APT.img windows.info.Info

Volatility 3 Framework 2.0.0

Variable	Value

Kernel Base	0x804d7000
DTB	0x319000
Symbols	file:///C:/Users/anon/Desktop/volatility3/volatility3/symbols/windows/ntkrnlpa.pdb/FD7B256CD2D14BC0B35BA9B89F691346-1.json.xz
Is64Bit	False
IsPAE	True
layer_name	0 WindowsIntelPAE
memory_layer	1 FileLayer
KdDebuggerDataBlock	0x80545b60
NTBuildLab	2600.xpsp_sp3_gdr.080814-1236
CSDVersion	3
KdVersionBlock	0x80545b38
Major/Minor	15.2600
MachineType	332
KeNumberProcessors	1
SystemTime	2009-05-05 19:28:57
NtSystemRoot	C:\WINDOWS
NtProductType	NtProductWinNt
NtMajorVersion	5
NtMinorVersion	1
PE MajorOperatingSystemVersion	5
PE MinorOperatingSystemVersion	1
PE Machine	332
PE TimeDateStamp	Thu Aug 14 09:33:12 2008


## Windows XP, cool. Now to analyze running processes

In [12]:
python ./vol.py -q -r csv -f ..\APT\APT.img windows.psscan.PsScan 

Volatility 3 Framework 2.0.0
"TreeDepth","PID","PPID","ImageFileName","Offset","Threads","Handles","SessionId","Wow64","CreateTime","ExitTime","File output"
0,464,704,"alg.exe","0x1f33628",6,105,0,False,"2009-04-16 16:10:21.000000 ",,"Disabled"
0,968,704,"svchost.exe","0x1fa4590",10,241,0,False,"2009-04-16 16:10:07.000000 ",,"Disabled"
0,1672,1624,"explorer.exe","0x1fa71a8",15,586,0,False,"2009-04-16 16:10:10.000000 ",,"Disabled"
0,796,884,"iexplore.exe","0x1fbdda0",8,152,0,False,"2009-05-05 19:28:28.000000 ",,"Disabled"
0,2004,1672,"VMwareUser.exe","0x1fc1a78",8,228,0,False,"2009-04-16 16:10:11.000000 ",,"Disabled"
0,1032,704,"VMwareService.e","0x1fc2570",3,175,0,False,"2009-04-16 16:10:16.000000 ",,"Disabled"
0,840,1672,"cmd.exe","0x204d648",1,33,0,False,"2009-05-05 15:56:24.000000 ",,"Disabled"
0,884,704,"svchost.exe","0x2054da0",17,208,0,False,"2009-04-16 16:10:07.000000 ",,"Disabled"
0,1212,704,"svchost.exe","0x2091da0",14,208,0,False,"2009-04-16 16:10:09.000000 ",,"Disabled"
0,20

## Lets make it a little prettier by omitting uninteresting data and pretty printing the csv output

In [13]:
python ./vol.py -q -r csv -f ..\APT\APT.img windows.psscan.PsScan | csvcut -c 2,3,4,6,7,10,11 | csvlook

Volatility 3 Framework 2.0.0
|   PID |  PPID | ImageFileName   | Threads | Handles |          CreateTime | ExitTime |
| ----- | ----- | --------------- | ------- | ------- | ------------------- | -------- |
|   464 |   704 | alg.exe         |       6 |     105 | 2009-04-16 16:10:21 |          |
|   968 |   704 | svchost.exe     |      10 |     241 | 2009-04-16 16:10:07 |          |
| 1,672 | 1,624 | explorer.exe    |      15 |     586 | 2009-04-16 16:10:10 |          |
|   796 |   884 | iexplore.exe    |       8 |     152 | 2009-05-05 19:28:28 |          |
| 2,004 | 1,672 | VMwareUser.exe  |       8 |     228 | 2009-04-16 16:10:11 |          |
| 1,032 |   704 | VMwareService.e |       3 |     175 | 2009-04-16 16:10:16 |          |
|   840 | 1,672 | cmd.exe         |       1 |      33 | 2009-05-05 15:56:24 |          |
|   884 |   704 | svchost.exe     |      17 |     208 | 2009-04-16 16:10:07 |          |
| 1,212 |   704 | svchost.exe     |      14 |     208 | 2009-04-16 16:10:09 |    

## In the last homework, we established that PID 796 "iexplore.exe" is suspicious due to it starting long after its parent process, PID 884 "svchost.exe". Lets explore it further by analyzing what handles it has opened so we can hypothesize what its doing

In [14]:
python .\vol.py -f ..\APT\APT.img windows.handles.Handles --pid 796

Volatility 3 Framework 2.0.0

PID	Process	Offset	HandleValue	Type	GrantedAccess	Name

796	iexplore.exe	0x821ace98	0xc	File	0x100020	\Device\HarddiskVolume1\WINDOWS\system32
796	iexplore.exe	0x82189028	0x578	File	0x12019f	\Device\NamedPipe\ROUTER
796	iexplore.exe	0x822eff10	0x57c	Event	0x1f0003	
796	iexplore.exe	0x81f229d0	0x580	Semaphore	0x100003	
796	iexplore.exe	0x81f22a08	0x584	Semaphore	0x100003	
796	iexplore.exe	0x81f30838	0x588	Semaphore	0x100003	
796	iexplore.exe	0x822eae40	0x58c	Semaphore	0x100003	
796	iexplore.exe	0xe1c29c90	0x590	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS
796	iexplore.exe	0xe1117fb8	0x594	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\NETBT\PARAMETERS\INTERFACES
796	iexplore.exe	0xe1c29c28	0x598	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\PARAMETERS
796	iexplore.exe	0xe1e9f728	0x59c	Key	0x20019	MACHINE\SYSTEM\CONTROLSET001\SERVICES\TCPIP\LINKAGE
796	iexplore.exe	0x821aaeb8	0x5a0	Semaphore	0x100003	
796	iexplore.exe	0x822

## That's a lot of information to sift through, and honestly I'm not even sure what I would be looking for. Additionally, this process seems to be doing a lot with cookies and temp files - normal web browser stuff, which doesn't look suspicious. For the same reason as before, lets look at DLLs instead.

In [15]:
python .\vol.py -q -r csv -f ..\APT\APT.img windows.dlllist.DllList --pid=796 | csvcut -c 2,6,7 | csvlook

Volatility 3 Framework 2.0.0
| PID | Name         | Path                                                                                                              |
| --- | ------------ | ----------------------------------------------------------------------------------------------------------------- |
| 796 | iexplore.exe | C:\Program Files\Internet Explorer\iexplore.exe                                                                   |
| 796 | ntdll.dll    | C:\WINDOWS\system32\ntdll.dll                                                                                     |
| 796 | kernel32.dll | C:\WINDOWS\system32\kernel32.dll                                                                                  |
| 796 | ADVAPI32.dll | C:\WINDOWS\system32\ADVAPI32.dll                                                                                  |
| 796 | RPCRT4.dll   | C:\WINDOWS\system32\RPCRT4.dll                                                                                    

## Aside from comctl32.dll, which is a benign dll used for GUI applications, none of these stand out (sigh). Lets see open network connections on the machine and use that to find a suspicious process. (I'm ignoring VMWare stuff because I'm assuming that was used to build this image)

In [22]:
python .\vol.py -f ..\APT\APT.img windows.cmdline.CmdLine

Volatility 3 Framework 2.0.0

PID	Process	Args

4	System	Required memory at 0x10 is not valid (process exited?)
564	smss.exe	\SystemRoot\System32\smss.exe
636	csrss.exe	C:\WINDOWS\system32\csrss.exe ObjectDirectory=\Windows SharedSection=1024,3072,512 Windows=On SubSystemType=Windows ServerDll=basesrv,1 ServerDll=winsrv:UserServerDllInitialization,3 ServerDll=winsrv:ConServerDllInitialization,2 ProfileControl=Off MaxRequestThreads=16
660	winlogon.exe	winlogon.exe
704	services.exe	C:\WINDOWS\system32\services.exe
716	lsass.exe	C:\WINDOWS\system32\lsass.exe
872	vmacthlp.exe	"C:\Program Files\VMware\VMware Tools\vmacthlp.exe"
884	svchost.exe	C:\WINDOWS\system32\svchost -k DcomLaunch
968	svchost.exe	C:\WINDOWS\system32\svchost -k rpcss
1088	svchost.exe	C:\WINDOWS\System32\svchost.exe -k netsvcs
1140	svchost.exe	C:\WINDOWS\system32\svchost.exe -k NetworkService
1212	svchost.exe	C:\WINDOWS\system32\svchost.exe -k LocalService
1512	spoolsv.exe	C:\WINDOWS\system32\spoolsv.exe
1672	explorer.exe

## MIRagent doesn't seem too suspicious to me. It's from Mandiant and it's executing what looks like a malware audit. None of these seem suspicious. Lets try running pslist with pscan, for if something is in pscan but not pslist, that process was hidden, which is suspicious.

In [25]:
python ./vol.py -q -r csv -f ..\APT\APT.img windows.pslist.PsList | csvcut -c 2,3,4,6,7,10,11 | csvlook

Volatility 3 Framework 2.0.0
|   PID |  PPID | ImageFileName   | Threads | Handles |          CreateTime | ExitTime |
| ----- | ----- | --------------- | ------- | ------- | ------------------- | -------- |
|     4 |     0 | System          |      55 |     254 |                     |          |
|   564 |     4 | smss.exe        |       3 |      19 | 2009-04-16 16:10:01 |          |
|   636 |   564 | csrss.exe       |      10 |     356 | 2009-04-16 16:10:06 |          |
|   660 |   564 | winlogon.exe    |      16 |     502 | 2009-04-16 16:10:06 |          |
|   704 |   660 | services.exe    |      15 |     254 | 2009-04-16 16:10:06 |          |
|   716 |   660 | lsass.exe       |      21 |     342 | 2009-04-16 16:10:06 |          |
|   872 |   704 | vmacthlp.exe    |       1 |      25 | 2009-04-16 16:10:07 |          |
|   884 |   704 | svchost.exe     |      17 |     208 | 2009-04-16 16:10:07 |          |
|   968 |   704 | svchost.exe     |      10 |     241 | 2009-04-16 16:10:07 |    

In [26]:
python ./vol.py -q -r csv -f ..\APT\APT.img windows.psscan.PsScan | csvcut -c 2,3,4,6,7,10,11 | csvlook

Volatility 3 Framework 2.0.0
|   PID |  PPID | ImageFileName   | Threads | Handles |          CreateTime | ExitTime |
| ----- | ----- | --------------- | ------- | ------- | ------------------- | -------- |
|   464 |   704 | alg.exe         |       6 |     105 | 2009-04-16 16:10:21 |          |
|   968 |   704 | svchost.exe     |      10 |     241 | 2009-04-16 16:10:07 |          |
| 1,672 | 1,624 | explorer.exe    |      15 |     586 | 2009-04-16 16:10:10 |          |
|   796 |   884 | iexplore.exe    |       8 |     152 | 2009-05-05 19:28:28 |          |
| 2,004 | 1,672 | VMwareUser.exe  |       8 |     228 | 2009-04-16 16:10:11 |          |
| 1,032 |   704 | VMwareService.e |       3 |     175 | 2009-04-16 16:10:16 |          |
|   840 | 1,672 | cmd.exe         |       1 |      33 | 2009-05-05 15:56:24 |          |
|   884 |   704 | svchost.exe     |      17 |     208 | 2009-04-16 16:10:07 |          |
| 1,212 |   704 | svchost.exe     |      14 |     208 | 2009-04-16 16:10:09 |    

## Nothing too crazy. Lets look at environment variables?

In [31]:
python ./vol.py -q -r csv -f ..\APT\APT.img windows.envars.Envars | csvcut | csvlook

Volatility 3 Framework 2.0.0
| TreeDepth |   PID | Process         | Block    | Variable                     | Value                                                                                         |
| --------- | ----- | --------------- | -------- | ---------------------------- | --------------------------------------------------------------------------------------------- |
|     False |   564 | smss.exe        | 0x110048 | Path                         | C:\WINDOWS\System32                                                                           |
|     False |   564 | smss.exe        | 0x110048 | SystemDrive                  | C:                                                                                            |
|     False |   564 | smss.exe        | 0x110048 | SystemRoot                   | C:\WINDOWS                                                                                    |
|     False |   564 | smss.exe        | 0x110048 | ve                           |

## Nothing too crazy here either, although it's cool to know this computers name is VICTIM1

## I ran out of time :/